![image.png](https://storage.googleapis.com/kaggle-competitions/kaggle/34349/logos/header.png?t=2022-03-09-00-33-57)

# 安装pyecharts用于绘图

In [ ]:
!pip install pyecharts

# 导入必要的包

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('talk')

from pyecharts import options as opts
from pyecharts.charts import Kline

# 读入对应的数据

In [ ]:
dataset_dir="../input/jpx-tokyo-stock-exchange-prediction/train_files/"
# 定义数据的路径
inputs = {
    "financials": f"{dataset_dir}/financials.csv",
    "options": f"{dataset_dir}/options.csv",
    "secondary_stock_prices": f"{dataset_dir}secondary_stock_prices.csv",
    "stock_prices": f"{dataset_dir}/stock_prices.csv",
    "trades": f"{dataset_dir}/trades.csv",
}

# 逐个读取数据到内存中
train = {}
for k, v in inputs.items():
    print(k)
    train[k] = pd.read_csv(v)

# 对1301号股票进行分析

In [ ]:
price = train["stock_prices"]

# 提取编号为1301的股票的价格信息
code = 1301
price_data = price[price["SecuritiesCode"] == code]
price_data

# 绘制编号为1301的股票的K线图

In [ ]:
# 定义画图的函数
def draw_kline(title, data):
    c = Kline()
    c.add_xaxis(np.array(data.index).tolist())
    c.add_yaxis(title, np.array(data).tolist())
    display(c.render_notebook())

stock_data = price_data.set_index('Date')[['Open', 'Close', 'Low', 'High']]
# 分别绘制2017年-2021年的股票K线图
for year in range(2017, 2022):
    draw_kline(f'{year}年上半年K线图', stock_data[f'{year}-01-01':f'{year}-06-01'])
    draw_kline(f'{year}年下半年K线图', stock_data[f'{year}-06-01':f'{year + 1}-01-01'])

In [ ]:
draw_kline('2017年-2021年总体K线图', stock_data)

# 移动平均线
+ 在股市用有各种各样的平均线，比如**周线、月线、年线**等，它们分别是**每周、每月、每年**工作日的**平均价格**
+ 这里展示了比较简单的平均线，分别是**5日线**、**25日线**和**75日线**，表示过去5个、25个、75个工作日的平均价格。
+ 然后使用移动化的方式展示平均线，与普通的平均线不同(如周线，开始日期一定是一个星期的第一个工作日)，移动化是从第x - period个工作日开始到第x个工作日的period个工作日的平均价格，**逐渐工作日递增x**，就像一个滑动窗口一样，逐工作日的滑过去。
+ 移动的具体原理如下图所示：
![](https://s3.bmp.ovh/imgs/2022/05/03/78f1e70bc27be03f.png)
![](https://s3.bmp.ovh/imgs/2022/05/03/0b7c654e258de39d.png)
+ 采用移动平均线的原因是，股票每个工作日的价格波动性可能是很大的，我们要尽量减少这种忽然的波动，从大局下手，掌握这支股票的大的变化趋势。
+ 移动平均线的特性正好可以展示出股票的逐渐变化的过程，纵观全局，凸显大的变化趋势，所以选择移动平均线作为我们的分析曲线。

In [ ]:
# 计算5日、25日和75日移动平均线
periods = [5, 25, 75]
cols = []
for period in periods:
    col = f"{period} windows simple moving average"
    # 设置最小周期为1，保证前面的不为NaN
    price_data[col] = price_data["Open"].rolling(period, min_periods=1).mean()
    cols.append(col)

# 绘制图像
fig, ax = plt.subplots(figsize=(20, 8))

for col in cols:
    ax.plot(price_data['Date'],price_data[col], label=col)
ax.set_ylabel("Stock Price")
ax.set_xlabel("Working Day")
plt.xticks([0, 200,400,600,800, 1000,1200])
ax.grid(True)
ax.legend()

+ 从上图中可以看出，总体的变化趋势是2017年到2018年中呈现上升趋势，2018年中达到最高点。2018年中到2020年中呈现下降趋势，跌幅较大。2020年中后有一定的平稳上升恢复的趋势。

# 投资回报率
+ 投资回报率是指一段时间内价格变化的比率，具体的计算方法为第x - period天与第x天的价格变化的百分比，表示在第x - period天投资，第x天时价格变动的百分比，这个百分比值越大，表示投资的回报越高
+ 下图展示了**5、25、75**个工作日内的投资回报率，从中可以看出**市场的潜力**与**市场发展的方向**，用于引导将来做出更好的投资时机选择

In [ ]:
periods = [5, 25, 75]
cols = []
for period in periods:
    col = f"{period} windows rate of return"
    price_data[col] = price_data["Open"].pct_change(period) * 100
    cols.append(col)

# 绘制回报率图
fig, ax = plt.subplots(figsize=(20, 8))

for col in cols:
    ax.plot(price_data['Date'],price_data[col], label=col)
ax.set_ylabel("Rate of Return (%)")
ax.set_xlabel("Date Time")
plt.xticks([0, 200,400,600,800, 1000,1200])
ax.grid(True)
ax.legend()

# 历史波动率
+ 历史波动率是一定时期内的股票价格波动率的评估。具体的计算方法是将所有的价格取对数后，进行逐项相差最后取标准差。
+ 历史波动率是风险评估指标之一，这个指标可以很好的指示股票价格变化的剧烈程度，具有很高的实际运用价值。
+ 一般而言，历史波动率比较大的股票适用于进行短期投资，历史波动率比较小的股票适合于长期持有。

In [ ]:
# 分别以 5、25 和 75 日为周期计算历史波动率
periods = [5, 25, 75]
cols = []
for period in periods:
    col = f"{period} windows volatility"
    price_data[col] = np.log(price_data["Open"]).diff().rolling(period).std()
    cols.append(col)

# 绘制图像
fig, ax = plt.subplots(figsize=(20, 8))

for col in cols:
    ax.plot(price_data['Date'],price_data[col], label=col)
ax.set_ylabel("Volatility")
ax.set_xlabel("Date Time")
plt.xticks([0, 200,400,600,800, 1000,1200])
ax.grid(True)
ax.legend()

# 填充缺失值
+ 对数据缺失值进行分析

In [ ]:
missing_sum = price_data.isna()
price_data['year'] = price_data.Date.str[:4]
missing_sum['year'] = price_data['year']
missing_sum = missing_sum.groupby("year").agg("sum")
print(missing_sum)
fig, ax = plt.subplots(figsize=(20, 5))
sns.heatmap(missing_sum, ax=ax)

+ 缺失值填充

In [ ]:
# 选择float64类型的列
price_data = price_data.select_dtypes(include=['float64', 'O'])

# 填充0
price_data = price_data.fillna(0)

# 特征量分析
## 设计平稳的特征量
+ 当我们在设计类似于股票价格这一类的时间序列数据的特征量时，平稳性是最重要的考虑因素。

+ 首先我们考虑使用原有的股票价格训练的情况，这样训练出来的模型会有什么样的结果：例如，假设模型训练期间的股票价格在100-110日元的范围内波动。如果将该值直接输入到模型中，模型会隐含地了解到股票价格在100-110日元的范围内波动。但是，这种隐含的假设在实际市场中是不成立的，如果股价在某一个时期飙升，该模型很大可能无法很好地工作。另一个特定于股票价格的例子是当股票价格范围由于股票拆分或反向股票拆分而大幅波动时，模型很大概率不会给出较好的结果。

+ 在接着考虑使用平稳特征的案例：例如，考虑20天的投资回报率，显然这个投资回报率不是正态分布，但可以发现除开某些市场动荡时期外，它近似于一个以0为期望的正态分布，它的特征是以0为中心的时间序列，例如上涨2%或下跌4%，并且可以预期在未来会有类似的分布。这正避免了模型学习到关于股价范围的不正确的隐含假设。

+ 当在可预见的将来内出现具有相似分布的特征时，我们就说这个特征量是稳定的。

+ 如果考虑到平稳性，那么在数据标准化过程中就多了许多需要注意的细节了。例如，我们在股票价格上做最大最小归一化，将股票价格的最小值和最大值分别映射到-1和1，看似很不错，但是这样不能保证训练出来的模型将将来的市场中依然适用，因为股票价格的最大最小值可能会改变，也可能在某一段时期内远远偏离这些值。

+ 因此在设计特征量时，平稳性是不可或缺的重要因素。

In [ ]:
# 对收盘价作特征分析
feats = price_data.copy()

# 投资回报率，计算方法同上
# 20个工作日的投资回报率
feats["return_1month"] = feats["Close"].pct_change(20)
# 40个工作日的投资回报率
feats["return_2month"] = feats["Close"].pct_change(40)
# 60个工作日的投资回报率
feats["return_3month"] = feats["Close"].pct_change(60)

# 收盘价波动情况，计算方法同历史变化率
# 20个工作日收盘价波动情况
feats["volatility_1month"] = np.log(feats["Close"]).diff().rolling(20).std()
# 40个工作日收盘价波动情况
feats["volatility_2month"] = np.log(feats["Close"]).diff().rolling(40).std()
# 60个工作日收盘价波动情况
feats["volatility_3month"] = np.log(feats["Close"]).diff().rolling(60).std()

# 当天收盘价与20个工作日的移动平均收盘价之间的偏差
feats["MA_gap_1month"] = feats["Close"] / feats["Close"].rolling(20).mean()
# 当天收盘价与40个工作日的移动平均收盘价之间的偏差
feats["MA_gap_2month"] = feats["Close"] / feats["Close"].rolling(40).mean()
# 当天收盘价与60个工作日的移动平均收盘价之间的偏差
feats["MA_gap_3month"] = feats["Close"] / feats["Close"].rolling(60).mean()

# 缺失值处理
feats = feats.fillna(0)
# 删除原始数据的列
feats = feats.drop(["Close"], axis=1)

# 定义绘图函数
def draw_feature(feature_name, col_name):
    fig, ax = plt.subplots(figsize=(20, 8))
    ax.plot(feats['Date'],feats[col_name])
    ax.set_ylabel(feature_name)
    ax.set_xlabel("Date Time")
    plt.xticks([0, 200,400,600,800, 1000,1200])
    ax.grid(True)
    ax.legend()
    
feats

## 绘制回报率图像

In [ ]:
draw_feature("Return Rate 1Month", "return_1month")
draw_feature("Return Rate 2Month", "return_2month")
draw_feature("Return Rate 3Month", "return_3month")

## 绘制收盘价格的波动率

In [ ]:
draw_feature("Volatility 1Month", "volatility_1month")
draw_feature("Volatility 2Month", "volatility_2month")
draw_feature("Volatility 3Month", "volatility_3month")

## 绘制收盘价的移动平均线

In [ ]:
draw_feature("Moving Average 1Month", "MA_gap_1month")
draw_feature("Moving Average 2Month", "MA_gap_2month")
draw_feature("Moving Average 3Month", "MA_gap_3month")